In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics

C:\Users\Ashish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
import os
path = "./Data/"
filename_read = os.path.join(path, "mpg.csv")
print(filename_read)

./Data/mpg.csv


In [21]:
# check the table structure
df = pd.read_csv(filename_read, na_values=['NA','?'])
df[0:5]

# separate the output column and impute the values
cars = df['name']
cars[1:10]

df.drop('name',1,inplace= True)

# imputing the missing values using the column missing
def missing_median(df,name):
    med = df[name].median()
    df[name] = df[name].fillna(med)

missing_median(df,'horsepower')

# convert a pandas dataframe to the x,y that tf needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find the type of the target column.
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type,'__iter__') else target_type
    print(target_type)
    # Encode to int for classification, float otherwise
    if target_type in (np.int64, np.int32):
        # classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.flaot32)
    else:
        # regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

    
x,y = to_xy(df,"mpg")

# initialize the model

model = Sequential()
model.add(Dense(25,
               input_dim = x.shape[1],
               activation='relu'))
model.add(Dense(10,
               activation='relu'))
model.add(Dense(1))

# compile model
model.compile(loss='mean_squared_error',
             optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

float64
Epoch 1/100
 - 7s - loss: 2763.5554
Epoch 2/100
 - 0s - loss: 600.0043
Epoch 3/100
 - 0s - loss: 336.8045
Epoch 4/100
 - 0s - loss: 245.8464
Epoch 5/100
 - 0s - loss: 184.1382
Epoch 6/100
 - 0s - loss: 151.0442
Epoch 7/100
 - 0s - loss: 142.2888
Epoch 8/100
 - 0s - loss: 127.9010
Epoch 9/100
 - 0s - loss: 117.5419
Epoch 10/100
 - 0s - loss: 108.9545
Epoch 11/100
 - 0s - loss: 103.5847
Epoch 12/100
 - 0s - loss: 96.4860
Epoch 13/100
 - 0s - loss: 91.6007
Epoch 14/100
 - 0s - loss: 91.9047
Epoch 15/100
 - 0s - loss: 83.5101
Epoch 16/100
 - 0s - loss: 81.9542
Epoch 17/100
 - 0s - loss: 76.4213
Epoch 18/100
 - 0s - loss: 74.1230
Epoch 19/100
 - 0s - loss: 72.2639
Epoch 20/100
 - 0s - loss: 72.1603
Epoch 21/100
 - 0s - loss: 72.3409
Epoch 22/100
 - 0s - loss: 68.0627
Epoch 23/100
 - 0s - loss: 64.0100
Epoch 24/100
 - 0s - loss: 73.7079
Epoch 25/100
 - 0s - loss: 64.0799
Epoch 26/100
 - 0s - loss: 60.9986
Epoch 27/100
 - 0s - loss: 59.2758
Epoch 28/100
 - 0s - loss: 58.1389
Epoch 29/

In [24]:
pred = model.predict(x)
print("shape: {}".format(pred.shape))
print(pred)

shape: (398, 1)
[[14.862509 ]
 [12.361349 ]
 [13.832693 ]
 [14.4193325]
 [14.27828  ]
 [ 7.9753323]
 [ 6.4458523]
 [ 6.805105 ]
 [ 6.431448 ]
 [ 9.524038 ]
 [11.368521 ]
 [11.791494 ]
 [11.125174 ]
 [ 9.200247 ]
 [24.165304 ]
 [20.69097  ]
 [20.729025 ]
 [21.641806 ]
 [25.041098 ]
 [28.427893 ]
 [24.182333 ]
 [23.703085 ]
 [24.651968 ]
 [22.470205 ]
 [21.059652 ]
 [ 9.428701 ]
 [11.733694 ]
 [10.717581 ]
 [12.377615 ]
 [25.438467 ]
 [23.664755 ]
 [24.547201 ]
 [25.900595 ]
 [19.90566  ]
 [18.996176 ]
 [18.908133 ]
 [19.352316 ]
 [19.33663  ]
 [11.848318 ]
 [ 9.886282 ]
 [12.801321 ]
 [13.491812 ]
 [ 9.033743 ]
 [ 9.62145  ]
 [ 8.644888 ]
 [18.564047 ]
 [25.04598  ]
 [18.848806 ]
 [19.37612  ]
 [23.758993 ]
 [24.40679  ]
 [27.584738 ]
 [25.671041 ]
 [28.277838 ]
 [28.696203 ]
 [27.904394 ]
 [27.452871 ]
 [25.311087 ]
 [25.952444 ]
 [29.03707  ]
 [25.017477 ]
 [24.927237 ]
 [12.111502 ]
 [10.603506 ]
 [13.965078 ]
 [13.093863 ]
 [14.563253 ]
 [ 8.155691 ]
 [12.433462 ]
 [12.366751 ]
 [10

In [26]:
# measure RMSE error
score = np.sqrt(metrics.mean_squared_error(pred,y))
print(f"final score (RMSE): {score}")

final score (RMSE): 3.9536452293395996


In [27]:
# sample predictions
for i in range(10):
    print(f"{i+1}. car name:{cars[i]}, MPG: {y[i]}, pred MPG: {pred[i]}")

1. car name:chevrolet chevelle malibu, MPG: [18.], pred MPG: [14.862509]
2. car name:buick skylark 320, MPG: [15.], pred MPG: [12.361349]
3. car name:plymouth satellite, MPG: [18.], pred MPG: [13.832693]
4. car name:amc rebel sst, MPG: [16.], pred MPG: [14.4193325]
5. car name:ford torino, MPG: [17.], pred MPG: [14.27828]
6. car name:ford galaxie 500, MPG: [15.], pred MPG: [7.9753323]
7. car name:chevrolet impala, MPG: [14.], pred MPG: [6.4458523]
8. car name:plymouth fury iii, MPG: [14.], pred MPG: [6.805105]
9. car name:pontiac catalina, MPG: [14.], pred MPG: [6.431448]
10. car name:amc ambassador dpl, MPG: [15.], pred MPG: [9.524038]
